# Importando as bibliotecas

In [1]:
import pandas as pd
import nltk

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
from datetime import datetime
from db import connection_db as conndb
from db import filters
from utils.plot_graphs import PlotGraphs
from utils.text_treatment import TextTreatment
from utils.text_vectorization import TextVectorization
from dateutil.relativedelta import relativedelta
from utils.nuvem_palavras import NuvemPalavras
from tqdm.auto import tqdm

tqdm.pandas()
from utils.busca_palavras import BuscaPalavras
from utils.facebook_prophet import FacebookProphet
from utils.estracao_interacao import ExtracaoInteracao

ERROR:prophet.plot:Importing plotly failed. Interactive plots will not work.


In [2]:
mongo_connection = conndb.MongoDBConnection(uri='mongodb://localhost:27017/', database_name='dadosVivamente',
                                            collection_name='dadosSemFiltros')
mongo_connection.connect()
collection = mongo_connection.collection

Conexão estabelecida com sucesso ao banco de dados.


In [3]:
collection_filters = filters.CollectionFilters(collection)
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')
collection_filters.apply_pipeline2(7, 2, 'posts7anos2anos')
collection_filters.apply_pipeline3('postsComBDIAndInfos')
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')
data_inicio = datetime(2017, 12, 1)
data_fim = data_inicio - relativedelta(months=2)
collection_filters.apply_pipeline5('postsFiltradosPorData', data_inicio, data_fim)
collection = collection_filters.collection

INFO:root:A coleção já existe: dadosComFiltrosIniciais
INFO:root:A coleção já existe: posts7anos2anos
INFO:root:A coleção já existe: postsComBDIAndInfos
INFO:root:A coleção já existe: postsComBDIAndInfosFiltroDataPosts
INFO:root:A coleção já existe: postsFiltradosPorData


In [4]:
print('Quantidade de Usuários com nível 2:', collection_filters.quant_users_cat('nivel', '$eq', 2))
print('Quantidade de Usuários do sexo Masculino com nível 2:',
      collection_filters.count_users_by_gender('nivel', '$eq', 2, 'M'))
print('Quantidade de Usuários do sexo Feminino com nível 2:',
      collection_filters.count_users_by_gender('nivel', '$eq', 2, 'F'))

Quantidade de Usuários com nível 2: 137
Quantidade de Usuários do sexo Masculino com nível 2: 18
Quantidade de Usuários do sexo Feminino com nível 2: 119


In [5]:
# Filtrando documentos com o atributo maior que 3
filtro = {"nivel": {"$eq": 2}}
documentos = collection.find(filtro)

In [6]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))
print(df.dtypes)

_id                         object
idade                        int64
sexo                        object
id_usuario                  object
nivel                      float64
pessimismo                  object
tristeza                    object
fracasso                    object
prazer                      object
culpa                       object
punicao                     object
estima                      object
critica                     object
suicida                     object
choro                       object
agitacao                    object
interesse                   object
indecisao                   object
desvalorizacao              object
energia                     object
sono                        object
irritabilidade              object
apetite                     object
concentracao                object
fadiga                      object
int_sexo                    object
quantAmigos                 object
postMessage                 object
postStory           

In [7]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

In [8]:
# Salvar o DataFrame em um arquivo CSV
df.to_csv('dados/com_filtros_datas/2meses/so_suicida_2_meses.csv', index=False)

In [9]:
extrator = ExtracaoInteracao(df)

# Extrair as interações e obter o DataFrame atualizado
df_com_interacoes = extrator.extract_interactions()

In [10]:
df.to_csv('dados/com_filtros_datas/2meses/interacoes.csv', index=False)

# Utilizando o Facebook Prophet para prever a quantidade de posts

In [ ]:
facebook_prophet = FacebookProphet()

facebook_prophet.prophet(df, 'postCreated_time')

# Plotar quantidade de posts por usuário

In [ ]:
plot_graphs = PlotGraphs()

# Agrupar por usuário, mês e ano
posts_grouped = df.groupby(['id_usuario', 'mes', 'ano']).size().reset_index(name='quantidade')

# Adicionar coluna com o período e converter para datetime
posts_grouped['periodo'] = pd.to_datetime(posts_grouped['mes'].astype(str) + '/' + posts_grouped['ano'].astype(str),
                                          format='%m/%Y')

# Deve retornar 0 se a conversão foi bem-sucedida.
print(posts_grouped['periodo'].isnull().sum())

posts_grouped.dtypes

In [ ]:
# plot_graphs.plot_posts_per_user(posts_grouped, 'dados/com_filtros_datas/2meses/graficos')

# Efentuando a limpeza dos dados

In [ ]:
tratamento_texto = TextTreatment()

In [ ]:
df['postMessageLimpo'] = df['postMessage'].fillna('').progress_apply(
    lambda texto: tratamento_texto.preprocessamento_texto(texto) if texto else '')

In [ ]:
# Exibir apenas as colunas postMessage e postMessageLimpo
df[['postMessage', 'postMessageLimpo']].head()

In [ ]:
busca_palavras = BuscaPalavras()

In [ ]:
resultado = busca_palavras.string_matching(df['postMessageLimpo'], 'dados/datasets/termos_depressivos_pt_br.txt')
# Salvar o resultado em um arquivo CSV
resultado.to_csv('dados/com_filtros_datas/2meses/so_suicida_termos_depressivos.csv')

resultados_fuzzy = busca_palavras.fuzzy_string_matching(df['postMessageLimpo'],
                                                        'dados/datasets/termos_depressivos_pt_br.txt')
df_resultados_fuzzy = pd.DataFrame(resultados_fuzzy,
                                   columns=['Palavra Procurada', 'Palavra Encontrada', 'Similaridade'])
# Salvar o resultado em um arquivo CSV
df_resultados_fuzzy.to_csv('dados/com_filtros_datas/2meses/so_suicida_termos_depressivos_fuzzy.csv', index=False)

In [ ]:
# Carregar stopwords para o TF-IDF e o Bag of Words
stop_words = tratamento_texto.get_stopwords()

In [ ]:
text_vectorizer = TextVectorization(df, stop_words)
# Vetorização TF-IDF
resultados_tfidf = text_vectorizer.tfidf_vectorization(
    'dados/com_filtros_datas/2meses/so_suicida_resultados_tfidf_unigramas.csv', 0.60, 2,
    (1, 1))

# Vetorização Bag of Words
resultados_bow = text_vectorizer.bag_of_words_vectorization(
    'dados/com_filtros_datas/2meses/so_suicida_resultados_bow_unigramas.csv',
    0.85, 5, (1, 1))

In [ ]:
text_vectorizer.plot_top_words(resultados_tfidf, column='score', title='Top 10 Palavras TF-IDF')
text_vectorizer.plot_top_words(resultados_bow, column='contagem', title='Top 10 Palavras Bag of Words')

In [ ]:
frequencias_bow = dict(zip(resultados_bow['palavra'], resultados_bow['contagem']))

# Gerar e plotar a nuvem de palavras
NuvemPalavras.plot_nuvem_palavras(frequencias_bow)

In [ ]:
frequencias_tfidf = dict(zip(resultados_tfidf['palavra'], resultados_tfidf['score']))

# Gerar e plotar a nuvem de palavras
NuvemPalavras.plot_nuvem_palavras(frequencias_tfidf)